In [252]:
import numpy as np
import scipy as sci
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

In [253]:
cols = ["rootstock", "girth4", "ext4", "girth15", "weight15"]
df = pd.read_table("data/T6_2_ROOT.DAT", header=None, sep="\s+", names=cols)

In [254]:
df.head()

,rootstock,girth4,ext4,girth15,weight15
0,1,1.11,2.569,3.58,0.760
1,1,1.19,2.928,3.75,0.821
2,1,1.09,2.865,3.93,0.928
3,1,1.25,3.844,3.94,1.009
4,1,1.11,3.027,3.60,0.766


In [255]:
k = len(set(df.rootstock))
N = df.shape[0]
ni = N // k

In [256]:
covs_df = df.groupby(by="rootstock").cov()
covs_df

girth4      ext4   girth15  weight15
rootstock                                                 
1         girth4    0.003393  0.020272  0.003739  0.001779
          ext4      0.020272  0.200691  0.058043  0.045790
          girth15   0.003739  0.058043  0.035184  0.028483
          weight15  0.001779  0.045790  0.028483  0.028293
2         girth4    0.003421  0.025773  0.008843  0.003233
          ext4      0.025773  0.304786  0.149834  0.083152
          girth15   0.008843  0.149834  0.115743  0.071106
          weight15  0.003233  0.083152  0.071106  0.056464
3         girth4    0.006850  0.031418  0.008700  0.006002
          ext4      0.031418  0.154296  0.047993  0.032861
          girth15   0.008700  0.047993  0.095086  0.067995
          weight15  0.006002  0.032861  0.067995  0.053393
4         girth4    0.007364  0.031384  0.014346  0.004366
          ext4      0.031384  0.262156  0.089652  0.039420
          girth15   0.014346  0.089652  0.083427  0.046557
          weight15  0.004366  0.039420  0.046557  0.030628
5         girth4    0.009914  0.057133  0.021943  0.009014
          ext4      0.057133  0.368078  0.154582  0.062531
          girth15   0.021943  0.154582  0.175707  0.097654
          weight15  0.009014  0.062531  0.097654  0.058549
6         girth4    0.014770  0.076387  0.021584  0.006626
          ext4      0.076387  0.444677  0.123270  0.037706
          girth15   0.021584  0.123270  0.107827  0.042727
          weight15  0.006626  0.037706  0.042727  0.018749

In [257]:
covs = [covs_df.xs(i+1) for i in range(k)]

In [258]:
Spl = 0
for i in range(k):
    Spl += covs[i] * ni
Spl /= (N - k)

In [259]:
Spl

,girth4,ext4,girth15,weight15
girth4,0.008707,0.046165,0.015077,0.005909
ext4,0.046165,0.330416,0.118738,0.057421
girth15,0.015077,0.118738,0.116757,0.067528
weight15,0.005909,0.057421,0.067528,0.046871


In [260]:
Spl_inv = np.linalg.inv(Spl)
Spl_inv

array([[490.04152282, -67.84396717, -39.74757884,  78.60408106],
       [-67.84396717,  14.56167035,  -4.0582595 ,  -3.43994539],
       [-39.74757884,  -4.0582595 ,  72.27416788, -94.14381807],
       [ 78.60408106,  -3.43994539, -94.14381807, 151.27418284]])

## A. Linear classification functions 

In [261]:
yhat = df.groupby("rootstock").mean().to_numpy()
yhat

array([[1.1375  , 2.977125, 3.73875 , 0.871125],
       [1.1575  , 3.109125, 4.515   , 1.2805  ],
       [1.1075  , 2.81525 , 4.455   , 1.391375],
       [1.0975  , 2.87975 , 3.90625 , 1.039   ],
       [1.08    , 2.55725 , 4.3125  , 1.181   ],
       [1.03625 , 2.214625, 3.59625 , 0.735   ]])

In [262]:
def Li(i, y):
    return yhat[i].T @ Spl_inv @ y - 0.5 * yhat[i].T @ Spl_inv @ yhat[i]

## B. Linear classification

In [263]:
def L(y):
    return np.array([Li(i, y) for i in range(k)])

def classify_linear(y):
    return np.argmax(L(y)) + 1

In [264]:
data = df.iloc[:, 1:]

In [265]:
pred = data.apply(classify_linear, axis=1)

In [267]:
confusion_matrix(df.rootstock, pred)

array([[5, 0, 0, 1, 0, 2],
       [0, 3, 2, 1, 2, 0],
       [0, 0, 6, 1, 1, 0],
       [3, 0, 1, 4, 0, 0],
       [0, 3, 1, 0, 3, 1],
       [2, 0, 0, 0, 2, 4]])

In [268]:
print(classification_report(df.rootstock, pred))

              precision    recall  f1-score   support

           1       0.50      0.62      0.56         8
           2       0.50      0.38      0.43         8
           3       0.60      0.75      0.67         8
           4       0.57      0.50      0.53         8
           5       0.38      0.38      0.38         8
           6       0.57      0.50      0.53         8

    accuracy                           0.52        48
   macro avg       0.52      0.52      0.52        48
weighted avg       0.52      0.52      0.52        48



## C. 

In [269]:
def Qi(i, y):
    return -0.5 * np.log(np.linalg.det(covs[i])) - 0.5 * (y - yhat[i]).T @ np.linalg.inv(covs[i]) @ (y - yhat[i])

In [270]:
def Q(y):
    return np.array([Qi(i, y) for i in range(k)])

def classify_quadratic(y):
    return np.argmax(Q(y)) + 1

In [271]:
data = df.iloc[:, 1:]

In [272]:
pred = data.apply(classify_quadratic, axis=1)

In [273]:
confusion_matrix(df.rootstock, pred)

array([[8, 0, 0, 0, 0, 0],
       [0, 7, 0, 1, 0, 0],
       [1, 0, 6, 0, 1, 0],
       [0, 0, 1, 7, 0, 0],
       [0, 3, 0, 0, 4, 1],
       [2, 0, 0, 0, 1, 5]])

In [274]:
print(classification_report(df.rootstock, pred))

              precision    recall  f1-score   support

           1       0.73      1.00      0.84         8
           2       0.70      0.88      0.78         8
           3       0.86      0.75      0.80         8
           4       0.88      0.88      0.88         8
           5       0.67      0.50      0.57         8
           6       0.83      0.62      0.71         8

    accuracy                           0.77        48
   macro avg       0.78      0.77      0.76        48
weighted avg       0.78      0.77      0.76        48



## D. Nearest-neighbour method

In [275]:
from sklearn.neighbors import KNeighborsClassifier

In [276]:
def dist(yi, yj):
    return (yi - yj).T @ Spl_inv @ (yi - yj)

In [293]:
classifier = KNeighborsClassifier(
    n_neighbors=3,
    metric=dist
)

In [294]:
classifier.fit(df.iloc[:, 1:], df.iloc[:, 0])

KNeighborsClassifier(metric=<function dist at 0x7f14b0bd9510>, n_neighbors=3)

In [295]:
pred = classifier.predict(df.iloc[:, 1:])

/home/evgeniy/.local/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [296]:
confusion_matrix(df.rootstock, pred)

array([[6, 0, 0, 2, 0, 0],
       [0, 7, 0, 0, 1, 0],
       [1, 0, 6, 0, 1, 0],
       [3, 0, 0, 5, 0, 0],
       [0, 1, 0, 0, 6, 1],
       [2, 0, 0, 1, 0, 5]])

In [297]:
print(classification_report(df.rootstock, pred))

              precision    recall  f1-score   support

           1       0.50      0.75      0.60         8
           2       0.88      0.88      0.88         8
           3       1.00      0.75      0.86         8
           4       0.62      0.62      0.62         8
           5       0.75      0.75      0.75         8
           6       0.83      0.62      0.71         8

    accuracy                           0.73        48
   macro avg       0.76      0.73      0.74        48
weighted avg       0.76      0.73      0.74        48



## E. Kernel density estimator classification

In [298]:
h = 1
p = 4
n = df.shape[0]

In [313]:
def fi(i, y):
    res = 0
    group = df[df.rootstock == i+1].to_numpy()[:, 1:]

    for j in range(ni):
        res += np.exp(-(y - group[j]).T @ Spl_inv @ (y-group[j]) / (2 * h ** 2))
    
    res /= n * h**p * np.linalg.det(Spl) ** 0.5 
    return res

In [322]:
def f(y):
    return np.array([fi(i, y) for i in range(k)])

def classify_kde(y):
    return np.argmax(f(y)) + 1

In [323]:
data = df.iloc[:, 1:]

In [324]:
pred = data.apply(classify_kde, axis=1)

In [325]:
confusion_matrix(df.rootstock, pred)

array([[8, 0, 0, 0, 0, 0],
       [0, 7, 1, 0, 0, 0],
       [1, 0, 6, 0, 1, 0],
       [1, 0, 0, 7, 0, 0],
       [0, 1, 0, 0, 6, 1],
       [3, 0, 0, 0, 0, 5]])

In [326]:
print(classification_report(df.rootstock, pred))

              precision    recall  f1-score   support

           1       0.62      1.00      0.76         8
           2       0.88      0.88      0.88         8
           3       0.86      0.75      0.80         8
           4       1.00      0.88      0.93         8
           5       0.86      0.75      0.80         8
           6       0.83      0.62      0.71         8

    accuracy                           0.81        48
   macro avg       0.84      0.81      0.81        48
weighted avg       0.84      0.81      0.81        48

